In [28]:
import pandas as pd
import requests as req

In [29]:
BASE_URL = "http://127.0.0.1:8000/"

In [30]:
resp = req.get(BASE_URL+"list_models")

In [31]:
resp

<Response [200]>

In [32]:
resp.json()

[{'name_model': 'log_reg',
  'description': 'Logistic regression ...',
  'hyperparams': [{'name': 'C',
    'description': 'L2 reg coeff.',
    'range': '0.001 - 1000'}]},
 {'name_model': 'rf',
  'description': 'Random Forest ...',
  'hyperparams': [{'name': 'n_estimators',
    'description': 'No. of base estimators.',
    'range': '100 - 500'},
   {'name': 'max_depth',
    'description': 'Max. depth of base estimator.',
    'range': '5 - 50'}]}]

In [33]:
df = pd.read_csv("Bank_Churn_Modelling.csv")

In [34]:
df.head(20)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2,134603.88,1,1,1,71725.73,0


In [35]:
df.isna().sum()

RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [36]:
data = df.values.tolist()

In [37]:
lr_input = {
    "data":data,
    "name_model": "lr",
     "hyp_range" :[
            { "name": "C",
             "min_value": -2,
            "max_value": 2,
            "steps": 5,
            "log_scale": True,
            "is_int": False
            }
     ]
}

In [38]:
resp = req.post(BASE_URL+"train_model", json=lr_input)

In [39]:
resp.json()

{'metric': 0.7414851502027642,
 'metric_name': 'f1_weighted',
 'best_params': {'model__C': 0.01}}

In [40]:
rf_input = {
    "data":data,
    "name_model": "rf",
     "hyp_range" :[
            { "name": "n_estimators",
             "min_value": 100,
            "max_value": 300,
            "steps": 3,
            "log_scale": False,
            "is_int": True
            },
        { "name": "max_depth",
           "min_value": 5,
            "max_value": 25,
            "steps": 5,
            "log_scale": False,
            "is_int": True
            }
        ]
}

In [41]:
resp = req.post(BASE_URL+"train_model", json=rf_input)

In [42]:
resp.json()

{'metric': 0.8481939527180336,
 'metric_name': 'f1_weighted',
 'best_params': {'model__max_depth': 15.0, 'model__n_estimators': 300.0}}

In [43]:
predict_lr_data = {
    "name_model": "lr",
    "CreditScore": 650,
    "Geography": "France",
    "Gender": "Female",
    "Age": 38,
    "Tenure": 5,
    "Balance": 76485,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 100900
}

In [44]:
resp = req.post(BASE_URL+"predict", json=predict_lr_data)

In [46]:
resp.json()

{'Exited_rate': 0.34611776205378025}

In [47]:
predict_rf_data = {
    "name_model": "rf",
    "CreditScore": 650,
    "Geography": "France",
    "Gender": "Female",
    "Age": 38,
    "Tenure": 5,
    "Balance": 76485,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 100900
}

In [48]:
resp = req.post(BASE_URL+"predict", json=predict_rf_data)

In [49]:
resp.json()

{'Exited_rate': 0.07162498443232226}

In [50]:
resp = req.delete(BASE_URL+"model_delete/lr")

In [51]:
resp.json()

{'message': 'lr was deleted successfully.'}

In [52]:
req.post(BASE_URL+"predict", json=predict_lr_data).json()

{'error': 'Firstly, the lr must be trained!'}

In [53]:
req.post(BASE_URL+"train_model", json=lr_input).json()

{'metric': 0.7414851502027642,
 'metric_name': 'f1_weighted',
 'best_params': {'model__C': 0.01}}

In [54]:
req.post(BASE_URL+"predict", json=predict_lr_data).json()

{'Exited_rate': 0.34611776205378025}